In [1]:
#module for creating table AAIDByMonth

%matplotlib inline
import pandas as pd
import sys
import numpy as np
import datetime
import seaborn as sns


In [3]:
from IPython.core.display import HTML
css = open('../Data/style-table.css').read() + open('../Data/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [4]:
USMTO_WHOLE = pd.read_csv('../Data/Intermediate.csv')

In [9]:
#Creates table by AAID and OrderDateFormatted

AAID_BY_MONTH = USMTO_WHOLE.groupby(['AAID','OrderDateFormatted'])['DaysLate','TotalSale','IsEmailOne', 'IsEmailTwo', 'IsCall', 'IsBackfill','Close','IP'].mean()
AAID_BY_MONTH.reset_index(level=1, inplace=True)
AAID_BY_MONTH.reset_index(level=1, inplace=True)
AAID_BY_MONTH_2 = USMTO_WHOLE.groupby(['AAID','OrderDateFormatted'])['Units','TotalSale'].sum()
AAID_BY_MONTH_2.reset_index(level=1, inplace=True)
AAID_BY_MONTH_2.reset_index(level=1, inplace=True)
AAID_BY_MONTH = AAID_BY_MONTH.merge(AAID_BY_MONTH_2, how = 'inner', on = ['AAID','OrderDateFormatted'])
AAID_BY_MONTH['TotalSaleAvg'] = AAID_BY_MONTH['TotalSale_x']
AAID_BY_MONTH['TotalSale'] = AAID_BY_MONTH['TotalSale_y']
AAID_BY_MONTH = AAID_BY_MONTH.drop('TotalSale_x', axis = 1)
AAID_BY_MONTH = AAID_BY_MONTH.drop('TotalSale_y', axis = 1)
USMTO_WHOLE.to_csv('../Data/AAID_BY_MONTH.csv')

In [10]:
#generates list of unique company ids

ALL_AAIDS = AAID_BY_MONTH.AAID.unique()

In [11]:
#gets rid of backfillers

AAID_BY_MONTH = AAID_BY_MONTH[AAID_BY_MONTH.DaysLate <= 90]

In [13]:
#loops through every company and creates new variables, which are all just the last month's or last three month's values

result = pd.DataFrame()

for a in ALL_AAIDS:
    tempdf = AAID_BY_MONTH[AAID_BY_MONTH.AAID == a]
    tempdf['UnitsLast'] = tempdf.Units.shift(1)
    tempdf['IsCallLast'] = tempdf.IsCall.shift(1)
    tempdf['TotalSaleAvgLast'] = tempdf.TotalSaleAvg.shift(1)
    tempdf['TotalSaleLastThree'] = (tempdf.TotalSale.shift(1) +tempdf.TotalSale.shift(2)  + tempdf.TotalSale.shift(3))/3
    tempdf['DaysLateLastThree'] = (tempdf.DaysLate.shift(1) +tempdf.DaysLate.shift(2)  + tempdf.DaysLate.shift(3))/3
    tempdf['DaysLateLast'] = tempdf.DaysLate.shift(1)
    tempdf.fillna(value = 0)
    result = result.append(tempdf)
    

C:\Users\lzhang\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lzhang\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lzhang\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [14]:
#removes blanks

result = result.fillna('')

In [15]:
#export to csv

result.to_csv('../Data/result.csv')